# Running a job

## Openfoam

We will run an example job using the OpenFoam 7 module for 

`OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7`

Lets load the module:
`module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7`

Source the OpenFoam bashrc:

```bash
source $foamDotFile```

Lets switch to our scratch space:

```bash
cd $SCRATCH
```


And copy over the run files:
``` bash
cp -r $WM_PROJECT_DIR/tutorials/multiphase/interFoam/laminar/damBreak ./```

An example run script:
    
```bash
#!/bin/bash
#SBATCH --account=prj3_phase1
#SBATCH --job-name=AMD
#SBATCH --output=AMD.%j.o
#SBATCH --error=AMD.%j.e
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --partition=spot-hbv3-16
#SBATCH --time=04:00:00

module purge
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/hbv3
. $foamDotFile
./Allrun
```

Lets switch into the folder 
```bash
cd damBreak
```

And after copying the above runscript into `run_job.slm` lets submit the job:

```bash
sbatch run_job.slm
```

## ANSYS

```bash
#!/bin/bash
#SBATCH --job-name=Free_Surf
#SBATCH --account=sysadmin
#SBATCH --partition=spot-hc-44
#SBATCH --time=2:00:00
#SBATCH --nodes=1
#SBATCH --tasks-per-node=44

## Match the number of licenses to the cores requested
#SBATCH --licenses=cfd_solve_level1@ansyslm:1,cfd_solve_level2@ansyslm:1,cfd_base@ansyslm:1

env | grep SLURM | sort -d
echo

export CASE="Free_Surf"

## Load the required modules
module load ansys/v212
module list

## Define files
export DEFFILE="${CASE}.def"

## Define additional options
OPTIONS="-double -batch"

CFX_PARALLEL="$CFX_BATCH_MODE $CFX_CCL_FILE $CFX_START_METHOD $CFX_PARTITION $CFX_PARTITIONER"

HOSTS=$(srun --ntasks-per-node $SLURM_CPUS_ON_NODE hostname | sort | uniq -c | awk '{print $2 "*" $1}' | paste -sd, -)
echo "HOSTS: $HOSTS"
echo ""

# Run CFX5SOLVE
cfx5solve -def Free_Surf.def ${OPTIONS} -start-method 'Intel MPI Distributed Parallel' -par-dist $HOSTS -partitioner ${CFXPARTITIONER}
```